<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Using OpenAI Functions API

The latest OpenAI models allow you to specify the model to output JSON results.
Langchain already had a lot of tools and prompts for this purpose, but OpenAI also released a slightly more fine-tuned and steerable version that can help always get JSON out.

https://platform.openai.com/docs/guides/function-calling

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from typing import Optional

from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage

In [13]:
# Choose the correct model based on: 
# https://platform.openai.com/docs/models/
llm = ChatOpenAI(model='gpt-4o-mini')

In [14]:
# We have a class we'd like to fill in with outputs from LLMs
# LLMs return text, but using Function Calling
# we can enforce some structure in the returns
class Scientist():
    def __init__(self,first_name,last_name):
        self.first_name = first_name
        self.last_name = last_name
        

In [15]:
# We define a JSON schema that specified
# the structure of the return data
json_schema = {
  "title": "Scientist",
  "description": "Information about a famous scientist",
  "type": "object",
  # properties is the dict used by OpenAI
  # we need to define the fields we want as shown here
  "properties": {
      "first_name": {
        'title':'First Name',
        'description': "First name of scientist",
        "type": "string"
      },
      "last_name":{
        'title':'Last Name',
        'description': "Last name of scientist",
        "type": "string"
      },
  },
  "required": ['first_name','last_name']
}

In [16]:
# Prompt
template = 'Name a famous {country} scientist'
#human_prompt = HumanMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_template(template)

In [17]:
# Now, we create the chain but enforce the structure in the chat/LLM
chain = chat_prompt|llm.with_structured_output(schema=json_schema)

In [18]:
result = chain.invoke({"country": "Indian"})

In [19]:
print(result) # {'first_name': 'A.P.J.', 'last_name': 'Abdul Kalam'}

{'first_name': 'C.V.', 'last_name': 'Raman'}


In [31]:
type(result) # dict

dict